In [ ]:
# Import some libraries

import numpy as np
from skimage import color, data, restoration
import matplotlib.pyplot as plt
import numpy.random as npr
import scipy
from scipy.signal import fftconvolve, convolve
from scipy.signal import convolve2d
import torch
import torch.nn as nn
from skimage import io
import skimage
from PIL import Image
from scipy import fftpack
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Define function to show images

def show_images(im1, im1_title, im2, im2_title, im3, im3_title, font):
    fig, (image1, image2, image3) = plt.subplots(1, 3, figsize=(15, 50))
    image1.imshow(im1, cmap='gray')
    image1.set_title(im1_title, fontsize=font)
    image1.set_axis_off()
    image2.imshow(im2, cmap='gray')
    image2.set_title(im2_title, fontsize=font)
    image2.set_axis_off()
    image3.imshow(im3, cmap='gray')
    image3.set_title(im3_title, fontsize=font)
    image3.set_axis_off()
    fig.subplots_adjust(wspace=0.02, hspace=0.2,
                                top=0.9, bottom=0.05, left=0, right=1)
    fig.show()

## Load the data

In [ ]:
#Load the target image
image = io.imread('./image.tif')

#Load the blurred and distorted images
blurred = io.imread('./blurred.tif')
distorted = io.imread('./distorted.tif')

#Load the kernel
psf = io.imread('./PSF.tif')

In [ ]:
show_images(image, 'Original image', blurred, 'Blurred image',\
           distorted, 'Blurred and noisy image', font=18)

## (Naive) Inverse filtering

Convolution of two signals (or images in our case) is equal to the pointwise multiplication of their Fourier transforms, i.e.

$y(i,j) = k(i,j) \ast x(i,j) + n(i,j)$ <--> $Y(u,v) = K(u,v)X(u,v) + N(u,v)$

Consequently, the simplest approach in deconvolution is to perform element-wise division in the Fourier domain to find the underlying image $\textit{x}$:

$\hat{X}(u,v) = \frac{Y(u,v)}{K(u,v)} = \frac{K(u,v)X(u,v) + N(u,v)}{K(u,v)} = \frac{K(u,v)X(u,v)}{K(u,v)} + \frac{N(u,v)}{K(u,v)}$, 

$\hat{x}(i,j) = F^{-1}\hat{X}(u,v)$

For this task let's firstly consider the case of an image corrupted with blur only.

In [ ]:
## SOLUTION
Y = scipy.fft.fft2(blurred)

# Let's do all necessary modifications with the PSF before applying Fourier transform:

kernel_size = (blurred.shape[0] - psf.shape[0], blurred.shape[1] - psf.shape[1])  
psf = np.pad(psf, (((kernel_size[0]+1)//2, kernel_size[0]//2), \
                    ((kernel_size[1]+1)//2, kernel_size[1]//2)), 'constant')
kernel = fftpack.ifftshift(psf)
K = scipy.fft.fft2(kernel)

X_hat = Y/K
x_hat = fftpack.ifft2(X_hat).real

In [ ]:
show_images(image, 'Original image', blurred, 'Blurred image',\
           x_hat, 'Restored with the inverse filter', font=18)

In real life the image is corrupted not only with blur, but also with noise, so next we are going to consider the inverse filtering of an image distorted with blur and noise.

In [ ]:
## SOLUTION
Y = scipy.fft.fft2(distorted)

# Do all necessary modifications with the PSF before applying Fourier transform:

kernel_size = (distorted.shape[0] - psf.shape[0], distorted.shape[1] - psf.shape[1])  
kernel = np.pad(psf, (((kernel_size[0]+1)//2, kernel_size[0]//2), ((kernel_size[1]+1)//2, kernel_size[1]//2)), 'constant')
# kernel = fftpack.ifftshift(kernel)
K = scipy.fft.fft2(kernel)

X_hat_noise = Y/K
x_hat_noise = fftpack.ifft2(X_hat_noise).real
x_hat_noise = np.abs(np.fft.ifftshift(np.fft.ifft2(np.fft.fftshift(X_hat_noise))))

In [ ]:
show_images(image, 'Original image', distorted, 'Blurred image',\
           x_hat_noise, 'Restored with inverse filter', font=18)

## Richardson-Lucy 

Image formation is described as following:
$\mathbf{y}_n = \mathcal{P}(\mathbf{Kx}_n)$, where $\mathcal{P}$ denotes Poisson noise disturbing the image, $\mathbf{y}_n, \mathbf{x}_n\ \text{and}\ \mathbf{K}$ denote vector representations of the data.

Richardson-Lucy deconvolution algotithm is based on the Bayes rule as following:
$p\left(\mathbf{y}_{n} \mid \mathbf{x}\right)=\frac{(\mathbf{K} \mathbf{x})_{n}^{\mathbf{y}_{n}} e^{-}(\mathbf{K} \mathbf{x})_{n}}{\mathbf{y}_{n} !}$

Maximization of probability $p\left(\mathbf{y}_{n} \mid \mathbf{x}\right)$ is equivalent to minimizartion of log probability

$\log (p(\mathbf{y} \mid \mathbf{x}))=\log (\mathbf{K} \mathbf{x})^{T} \mathbf{y}-(\mathbf{K} \mathbf{x})^{T} \mathbf{1}-\sum_{i=1}^{M} \log \left(\mathbf{y}_{n} !\right)$

Three main conditions:
- $J = -log(p(\mathbf{y}|\mathbf{x})), \nabla{J} = 0$ - at solution gradient is zero;

- $\frac{\mathbf{x}_{i+1}}{\mathbf{x}} = 1$ - when converged, further iterations do not change;

- PSF is normalized such that sum of elements equals to 1.

We can form the Richardson-Lucy multiplicative update rule:
$\mathbf{x}_{i+1}=\mathbf{K}^{T} \odot(\mathbf{K} \mathbf{x})^{-1} \odot \mathbf{y} \odot \mathbf{x}_{i}$

In [ ]:
im_deconv = distorted.copy()

#Set number of Richardson-Lucy iterations
iterations = 10

for _ in range(iterations):
    print('Running Richardson-Lucy iteration {}'.format(_+1))
    # Calculate Kx
    Kx = convolve2d(im_deconv, psf[::-1, ::-1], mode='same', boundary='symm')
    # Calculate (Kx)^-1 (*) y
    relative_blur = distorted/Kx
    # Calculate K^T(*) (Kx)^-1 (*) y
    KTx = convolve2d(relative_blur, psf[::-1, ::-1], mode='same', boundary='symm')
    im_deconv = KTx * im_deconv

In [ ]:
show_images(image, 'Original image', distorted, 'Blurred and noisy image',\
           im_deconv, 'Restored with Richardson-Lucy', font=18)

#### Include Total-Variation regularization 

Total-variation implies regularization of the gradient of the image. It is based on the principle that signals with excessive and possibly spurious detail have high total variation (c) Wikipedia.

With inclusion of a regularization the solution yields the form

$J = -(\log (p(\mathbf{y} \mid \mathbf{x}))=\log (\mathbf{K} \mathbf{x})^{T} \mathbf{y}-(\mathbf{K} \mathbf{x})^{T} \mathbf{1}-\sum_{i=1}^{M} \log \left(\mathbf{y}_{n} !\right)) + \lambda|\nabla\mathbf{x}|$.

Finally, the Richardson-Lucy multiplicative update rule has the shape of

$\mathbf{x}_{i+1}=\mathbf{K}^{T} \odot(\mathbf{K} \mathbf{x})^{-1} \odot \mathbf{y} \odot \frac{\mathbf{x}_{i}}{1 - \lambda \text{div}\left( \frac{\nabla{\mathbf{x}}}{|\nabla{\mathbf{x}}|} \right)}$.

In [ ]:
def tv_grad(image, lamda):
    
    #Let's calculate image gradients over x and y axis (\nabla{x})
    dx = np.gradient(image)[1] 
    dy = np.gradient(image)[0]
    
    #We add small epsilon value to avoid numerical instability
    eps = 1e-6
    
    norm = np.sqrt(dx*dx + dy*dy) #|\nabla{x}|
    ndx = dx.copy()
    ndy = dy.copy()
    
    ndx[norm<eps] = eps
    ndy[norm<eps] = eps
    
    #Calculate (\nabla{x}/|\nabla{x}|) over x and y axis
    ndx[norm>=eps] = dx[norm>=eps] / norm[norm>=eps]
    ndy[norm>=eps] = dy[norm>=eps] / norm[norm>=eps]
    
    #Calculate gradients of the equation above over x and y axis
    ddx = np.gradient(ndx)[1] 
    ddy = np.gradient(ndx)[0] 

    #Calculate TV-part of the equation
    tv_part = 1/(1 - (ddx + ddy)*lamda)
    del ndx, ndy, ddx, ddy, dx, dy
    
    return tv_part

In [ ]:
im_deconv = distorted.copy()

#Set number of Richardson-Lucy iterations
iterations = 10

for _ in range(iterations):
    print('Running Richardson-Lucy iteration {}'.format(_+1))
    # Calculate Kx
    Kx = convolve2d(im_deconv, psf[::-1, ::-1], mode='same', boundary='symm')
    # Calculate (Kx)^-1 (*) y
    relative_blur = distorted/Kx
    # Calculate K^T(*) (Kx)^-1 (*) y
    KTx = convolve2d(relative_blur, psf[::-1, ::-1], mode='same', boundary='symm')
    im_deconv = tv_grad(im_deconv, 1e-1)* KTx * im_deconv

In [ ]:
show_images(image, 'Original image', distorted, 'Blurred image',\
           im_deconv, 'Restored with Richardson-Lucy with TV', font=18)